# Guía 3: SQL Avanzado

En esta guía usted deberá realizar consultas SQL en un servidor virtual con PostgreSQL que contiene
datos de películas extraídos de IMDb. El esquema de los datos es el siguiente:

* $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\underline{\text{anho}}}, \color{blue}{\text{calificacion}})$ 
* $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{nombre}}}, \color{blue}{\text{genero}})$
* $\color{green}{\textbf{personaje}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}},  \color{blue}{\underline{\text{a_nombre}}}, \color{blue}{\underline{\text{personaje}}})$

La tabla $\color{green}{\textbf{personaje}}$ usa llaves foráneas que hacen referencia a las tablas de $\color{green}{\textbf{actor}}(\color{blue}{\underline{\text{a_nombre}}})$ y $\color{green}{\textbf{pelicula}}(\color{blue}{\underline{\text{p_nombre}}}, \color{blue}{\underline{\text{p_anho}}})$.

Para iniciar el servidor virtual, instalar la base de datos postgres, y descargar los datos e importarlos, debe correr el siguiente bloque:



In [1]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# pin package versions for google colab compatibility
!pip install SQLAlchemy==1.4.46
!pip install ipython-sql==0.4.1
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
%sql create schema guia;
!wget -cO - https://github.com/IIC2413/Syllabus-2023-1/raw/main/Guias/extra/guia2.sql > /content/guia2.sql
!psql postgres < /content/guia2.sql
%sql set search_path to guia;

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,028 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,590 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:

""


Ejecute la siguiente consulta para probar que todo ande bien:

In [2]:
%sql SELECT * FROM pelicula;

 * postgresql+psycopg2://@/postgres


,nombre,anho,calificacion
0,The Shawshank Redemption,1994,9.2
1,The Godfather,1972,9.2
2,The Godfather: Part II,1974,9.0
3,The Dark Knight,2008,8.9
4,12 Angry Men,1957,8.9
...,...,...,...
245,The King's Speech,2010,8.0
246,The Avengers,2012,8.0
247,Lagaan: Once Upon a Time in India,2001,8.0
248,Beauty and the Beast,1991,8.0


Para ejecutar consultas multi-lineas use el tag %%sql:

In [ ]:
%%sql
SELECT * FROM 
pelicula;

 * postgresql+psycopg2://@/postgres


,nombre,anho,calificacion
0,The Shawshank Redemption,1994,9.2
1,The Godfather,1972,9.2
2,The Godfather: Part II,1974,9.0
3,The Dark Knight,2008,8.9
4,12 Angry Men,1957,8.9
...,...,...,...
245,The King's Speech,2010,8.0
246,The Avengers,2012,8.0
247,Lagaan: Once Upon a Time in India,2001,8.0
248,Beauty and the Beast,1991,8.0



## Pregunta 1
La cantidad de películas

In [ ]:
%%sql
SELECT COUNT(*) AS conteo 
FROM pelicula;

 * postgresql+psycopg2://@/postgres


,conteo
0,250


##Pregunta 2
La cantidad de años distintos de estreno de las películas

In [ ]:
%%sql
SELECT COUNT(DISTINCT anho) AS conteo 
FROM pelicula;

 * postgresql+psycopg2://@/postgres


,conteo
0,81


##Pregunta 3
Los nombres, años y calificaciones de las 10 películas mejor calificadas, ordenadas por calificación descendente, luego por nombre ascendente y luego por año ascendente (solo hay que devolver 10 tuplas, incluso si hay empates de calificación).

In [ ]:
%%sql
SELECT nombre, anho, calificacion 
FROM pelicula 
ORDER BY calificacion DESC, nombre, anho 
LIMIT 10;

 * postgresql+psycopg2://@/postgres


,nombre,anho,calificacion
0,The Godfather,1972,9.2
1,The Shawshank Redemption,1994,9.2
2,The Godfather: Part II,1974,9.0
3,12 Angry Men,1957,8.9
4,"Il buono, il brutto, il cattivo",1966,8.9
5,Pulp Fiction,1994,8.9
6,Schindler's List,1993,8.9
7,The Dark Knight,2008,8.9
8,The Lord of the Rings: The Return of the King,2003,8.9
9,Fight Club,1999,8.8


##Pregunta 4
Los nombres de los actores y las actrices que aparecen en las películas de la Pregunta 3. Cada actor/actriz solo debe aparecer en los resultados una vez.


In [ ]:
%%sql
SELECT DISTINCT a_nombre 
FROM personaje
  WHERE (p_nombre,p_anho) IN (
    SELECT nombre, anho 
    FROM pelicula 
    ORDER BY calificacion DESC, nombre, anho 
    LIMIT 10 
  );

 * postgresql+psycopg2://@/postgres


,a_nombre
0,"Abbott, Jane (II)"
1,"Acevedo, Gino (I)"
2,"Aiello, Danny"
3,"Ajala, David"
4,"Alagich, Richard"
...,...
867,"Zahir, Essa"
868,"Zahrn, Will"
869,"Zaideman, Kevin"
870,"Ziehm, Rainer"


##Pregunta 5
Los nombres de solo las actrices de las películas de la Pregunta 4. Cada actriz debe aparecer en los resultados solo una vez. 

In [ ]:
%%sql
SELECT DISTINCT a_nombre FROM personaje
  WHERE (p_nombre,p_anho) IN (
    SELECT nombre,anho 
    FROM pelicula 
    ORDER BY calificacion DESC, nombre, anho 
    LIMIT 10 
  ) AND a_nombre IN (
	SELECT nombre 
	FROM actor 
	WHERE genero='F'
  );

 * postgresql+psycopg2://@/postgres


,a_nombre
0,"Abbott, Jane (II)"
1,"Aldaz, Tracy L."
2,"Allman, Cie"
3,"Alonso, Chelo"
4,"Arquette, Alexis"
...,...
160,"Wolf, Lisa (IV)"
161,"Wyrobiec-Bank, Lidia"
162,"Yerushalmi, Esti"
163,"Yohn, Erica"


##Pregunta 6
Las distintas décadas de las películas en las cuales apareció Clint Eastwood. (Tiene tres resultados; uno es 1960.)

HINT: En el SELECT pueden ocupar operaciónes aritmeticas. Por ejemplo, pueden escribir 'SELECT año*10 + edad'.

In [ ]:
%%sql
SELECT DISTINCT p_anho - p_anho%10 AS decada
FROM personaje 
WHERE a_nombre='Eastwood, Clint';

 * postgresql+psycopg2://@/postgres


,decada
0,1960
1,1990
2,2000


##Pregunta 7
El conteo de películas distintas por año (para cada año en la tabla pelicula, devolver su conteo de películas).

In [ ]:
%%sql
SELECT anho, COUNT(DISTINCT (nombre, anho)) AS conteo -- DISTINCT no es necesario, pero está bien
FROM pelicula 
GROUP BY anho;

 * postgresql+psycopg2://@/postgres


,anho,conteo
0,1921,1
1,1925,1
2,1926,1
3,1927,2
4,1928,1
...,...,...
76,2012,5
77,2013,4
78,2014,8
79,2015,7


##Pregunta 8
Devuelva los mismos resultados como en la Pregunta 7, pero ordenados por conteo descendente y borrando los años cuyo conteo sea menor que 2.

In [ ]:
%%sql
SELECT anho, COUNT(DISTINCT(nombre,anho)) AS conteo
FROM pelicula 
GROUP BY anho 
HAVING COUNT(nombre) >1 
ORDER BY conteo DESC;

 * postgresql+psycopg2://@/postgres


,anho,conteo
0,1995,9
1,1957,8
2,2014,8
3,2001,7
4,2015,7
...,...,...
57,1992,2
58,1927,2
59,1964,2
60,1963,2


##Pregunta 9
Sea una buena película una película con una calificación $\geq 8{,}6$. Para cada actor/actriz, cuente sus distintas buenas películas. Si un actor/una actriz no tiene una buena película, se puede omitirlo/la de los resultados. Ordene los resultados por conteo descendente.

In [ ]:
%%sql
SELECT a_nombre, COUNT(DISTINCT(nombre,anho)) AS conteo
FROM personaje
JOIN pelicula
  ON nombre = p_nombre 
    AND anho = p_anho
    AND calificacion > 8.5
GROUP BY a_nombre
ORDER BY conteo DESC;

 * postgresql+psycopg2://@/postgres


,a_nombre,conteo
0,"Weaving, Hugo",4
1,"Astin, Sean",3
2,"Monaghan, Dominic",3
3,"Lee, Christopher (I)",3
4,"Boyd, Billy (I)",3
...,...,...
2132,"Bates, Billy (I)",1
2133,"Batista, Diego",1
2134,"Batista, Leandro Dias",1
2135,"Bau, Joseph",1


##Pregunta 10
Cuente el número de actrices distintas por película. De haber una película sin actrices, deje un conteo de 0 (en vez de omitir la película).

In [ ]:
%%sql 
SELECT p_nombre, COUNT(DISTINCT(nombre)) AS conteo
FROM personaje
  LEFT JOIN actor
    ON a_nombre = nombre AND genero = 'F'
GROUP BY p_nombre
ORDER BY conteo DESC;

 * postgresql+psycopg2://@/postgres


,p_nombre,conteo
0,The Wolf of Wall Street,94
1,Catch Me If You Can,66
2,Singin' in the Rain,65
3,Harry Potter and the Deathly Hallows: Part 2,55
4,The Dark Knight Rises,55
...,...,...
245,Paths of Glory,1
246,Lawrence of Arabia,1
247,The Thing,1
248,The Great Escape,0


## Pregunta 11

Nombre y año de la película con la mayor cantidad de actores apareciendo en esta película. Si un actor interpreta dos personajes distintos en una película, se considera solo una. Puede existir más de una película con esta propiedad.

In [ ]:
%%sql

SELECT p_nombre, p_anho
FROM personaje
GROUP BY p_nombre, p_anho
HAVING COUNT(DISTINCT a_nombre) = (
                  SELECT MAX(headcount)
                  FROM (
                        SELECT p_nombre, p_anho, COUNT(DISTINCT a_nombre) AS headcount
                        FROM personaje
                        GROUP BY p_nombre, p_anho
                        ORDER BY headcount DESC
                  ) AS Conteos
)

 * postgresql+psycopg2://@/postgres


,p_nombre,p_anho
0,The Dark Knight Rises,2012
